In [3]:
%pip install -q --upgrade qiskit>=1.2.0 qiskit-algorithms>=0.3.1 qiskit-machine-learning>=0.8.3 qiskit-aer scikit-learn>=1.3.0 pandas>=2.0.0 matplotlib
import sys 
print(f"✅ Installation complete!") 
print(f"🐍 Python version: {sys.version_info.major}.{sys.version_info.minor}") 

Note: you may need to restart the kernel to use updated packages.
✅ Installation complete!
🐍 Python version: 3.12



[notice] A new release of pip is available: 24.2 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [4]:
%pip install --upgrade qiskit-aer


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.2 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [5]:
# CELL 1:  IMPORTS

import warnings
warnings.filterwarnings("ignore")

import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split, StratifiedShuffleSplit
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.decomposition import PCA
from sklearn.feature_selection import SelectKBest, f_classif
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from sklearn.svm import SVC

# Qiskit imports 
from qiskit import QuantumCircuit
from qiskit.circuit import ParameterVector
from qiskit.circuit.library import RZZGate, ZZFeatureMap, ZFeatureMap
from qiskit_aer import AerSimulator

from qiskit_machine_learning.kernels import FidelityQuantumKernel
from qiskit_machine_learning.algorithms import QSVC

from qiskit_algorithms.utils import algorithm_globals

# Reproducibility
SEED = 42
np.random.seed(SEED)
algorithm_globals.random_seed = SEED

print("✅ All imports successful!")


✅ All imports successful!


In [6]:
# CELL 2: 
CSV_PATH = "hybrid_phishing_dataset_raw.csv"   # path to your CSV
SMALL_SUBSET = True     
SUBSET_SIZE = 300          
K_QUBITS = 10              


In [7]:
# CELL 3: LOAD CSV + INITIAL PREPROCESSING
df = pd.read_csv(CSV_PATH)
print("Original rows:", len(df))

# drop obvious unused columns if present
for c in ["web_Unnamed: 0", "email_label"]:
    if c in df.columns:
        df = df.drop(columns=[c])

# Ensure final_label exists and numeric
if "final_label" not in df.columns:
    raise ValueError("final_label column not found in CSV. Please provide final_label (0/1).")
df = df.reset_index(drop=True)

# Subsample (stratified) if SMALL_SUBSET True to speed up experimentation
if SMALL_SUBSET:
    sss = StratifiedShuffleSplit(n_splits=1, train_size=SUBSET_SIZE, random_state=SEED)
    for idx, _ in sss.split(df, df["final_label"]):
        df = df.iloc[idx].reset_index(drop=True)
    print(f"Subsampled to {len(df)} rows for small-scale experiment.")

# Domain trust (frequency-based)
domain_freq = df["email_from_domain"].value_counts(normalize=True)
df["email_from_domain_score"] = df["email_from_domain"].map(lambda x: 1 - domain_freq.get(x, 0))

# TLD risk (heuristic)
tld_risk = {"com": 0.1, "org": 0.1, "net": 0.3}
df["web_tld_score"] = df["web_tld"].map(lambda x: tld_risk.get(x, 0.7)).astype(float)

# IP to integer (safe)
def ip_to_int(ip):
    try:
        parts = str(ip).split(".")
        if len(parts) != 4:
            return 0
        return (int(parts[0]) << 24) + (int(parts[1]) << 16) + (int(parts[2]) << 8) + int(parts[3])
    except:
        return 0

df["web_ip_score"] = df["web_ip_add"].apply(ip_to_int).astype(float)

# HTTPS binary
df["web_https_bin"] = df["web_https"].map({"yes": 1, "no": 0}).fillna(0).astype(float)

# Content length
df["content_len"] = df["web_content"].fillna("").apply(len).astype(float)

print("Processed rows:", len(df))
df.head(1).T


Original rows: 8000
Subsampled to 300 rows for small-scale experiment.
Processed rows: 300


,0
email_subject_len,49
email_has_urgent_keyword,1
email_from_domain,promo.websites.com>
email_url_len,0
email_num_links_in_email,0
web_url,http://ddral.20m.com/
web_url_len,21
web_ip_add,163.160.79.209
web_geo_loc,United Kingdom
web_tld,com


In [8]:
# CELL 4: TRAIN/TEST SPLIT 
y_all = df["final_label"].values
train_df, test_df = train_test_split(df, test_size=0.2, random_state=SEED, stratify=y_all)


train_df = train_df.reset_index(drop=True)
test_df  = test_df.reset_index(drop=True)

print("Train rows:", len(train_df), "Test rows:", len(test_df))


Train rows: 240 Test rows: 60


In [9]:
# CELL 5: SEMANTIC PCA for q_* features -> 2 components 
semantic_cols = ["q_semantic_similarity", "q_domain_trust", "q_sentiment_interference", "q_url_entropy"]
# fill missing
train_sem = train_df[semantic_cols].fillna(0).values
test_sem  = test_df[semantic_cols].fillna(0).values

sem_pca = PCA(n_components=2, random_state=SEED)
sem_pca.fit(train_sem)

train_sem_comp = sem_pca.transform(train_sem)
test_sem_comp  = sem_pca.transform(test_sem)

train_df["sem_comp_0"] = train_sem_comp[:, 0]
train_df["sem_comp_1"] = train_sem_comp[:, 1]
test_df["sem_comp_0"]  = test_sem_comp[:, 0]
test_df["sem_comp_1"]  = test_sem_comp[:, 1]

print("Explained variance ratio (semantic PCA):", sem_pca.explained_variance_ratio_)


Explained variance ratio (semantic PCA): [0.86863115 0.12137973]


In [10]:
# CELL 6: BUILD MAPPED FEATURES (Q0..Q9) and SCALE (fit scaler on train)
mapped_cols_train = pd.DataFrame({
    # Group A (Q0 - Q3)
    "Q0_subject_len": train_df["email_subject_len"].fillna(0).astype(float),
    "Q1_urgent": train_df["email_has_urgent_keyword"].fillna(0).astype(float),
    "Q2_url_len": train_df["email_url_len"].fillna(0).astype(float),
    "Q3_num_links": train_df["email_num_links_in_email"].fillna(0).astype(float),

    # Group B (Q4 - Q7)
    "Q4_domain_age": train_df["domain_age"].fillna(0).astype(float),
    "Q5_https": train_df["web_https_bin"].fillna(0).astype(float),
    "Q6_tld_score": train_df["web_tld_score"].fillna(0).astype(float),
    "Q7_ip_score": train_df["web_ip_score"].fillna(0).astype(float),

    # Group C compressed (Q8 - Q9)
    "Q8_sem_comp0": train_df["sem_comp_0"].astype(float),
    "Q9_sem_comp1": train_df["sem_comp_1"].astype(float)
})

mapped_cols_test = pd.DataFrame({
    "Q0_subject_len": test_df["email_subject_len"].fillna(0).astype(float),
    "Q1_urgent": test_df["email_has_urgent_keyword"].fillna(0).astype(float),
    "Q2_url_len": test_df["email_url_len"].fillna(0).astype(float),
    "Q3_num_links": test_df["email_num_links_in_email"].fillna(0).astype(float),

    "Q4_domain_age": test_df["domain_age"].fillna(0).astype(float),
    "Q5_https": test_df["web_https_bin"].fillna(0).astype(float),
    "Q6_tld_score": test_df["web_tld_score"].fillna(0).astype(float),
    "Q7_ip_score": test_df["web_ip_score"].fillna(0).astype(float),

    "Q8_sem_comp0": test_df["sem_comp_0"].astype(float),
    "Q9_sem_comp1": test_df["sem_comp_1"].astype(float)
})

# Fit scaler on training mapped columns
scaler = MinMaxScaler()
X_train_10 = scaler.fit_transform(mapped_cols_train.values)
X_test_10  = scaler.transform(mapped_cols_test.values)

y_train = train_df["final_label"].values
y_test  = test_df["final_label"].values

print("Shape X_train_10:", X_train_10.shape, "X_test_10:", X_test_10.shape)


Shape X_train_10: (240, 10) X_test_10: (60, 10)


In [11]:
# CELL 8: PARAMETERIZED CUSTOM FEATURE MAP (10 params) 
num_qubits = 10
params = ParameterVector("x", length=num_qubits)

def build_phish_feature_map(param_vector: ParameterVector) -> QuantumCircuit:
    qc = QuantumCircuit(num_qubits, name="PhishFM")
    x = param_vector

    # Single-qubit encodings (axes as specified)
    qc.ry(np.pi * x[0], 0)      # Q0 = email_subject_len -> RY
    qc.rz(np.pi * x[1], 1)      # Q1 = urgent -> RZ(pi * bitlike)
    qc.rx(np.pi * x[2], 2)      # Q2 = url_len -> RX
    qc.ry(np.pi * x[3], 3)      # Q3 = num_links -> RY

    qc.ry(np.pi * x[4], 4)      # Q4 = domain_age -> RY
    qc.rz(np.pi * x[5], 5)      # Q5 = https -> RZ(pi * bit)
    qc.rx(np.pi * x[6], 6)      # Q6 = tld_score -> RX
    qc.ry(np.pi * x[7], 7)      # Q7 = ip_score -> RY

    qc.ry(np.pi * x[8], 8)      # Q8 = sem_comp0 -> RY
    qc.rx(np.pi * x[9], 9)      # Q9 = sem_comp1 -> RX

    # -----------------------
    # Group A entanglement (content)
    # CZ(0,1) + RZZ(x0*x1)
    qc.cz(0,1)
    qc.append(RZZGate(x[0] * x[1]), [0,1])

    # CX(1,2) + RYY(x2*x3)
    qc.cx(1,2)
    qc.ryy(x[2] * x[3], 2, 3)

    # -----------------------
    # Group B entanglement (domain trust)
    qc.cz(4,5)
    qc.cz(5,6)
    qc.append(RZZGate(x[4] * x[5]), [4,5])   # RZZ(domain_age * https)
    qc.rxx(x[7] * x[6], 7, 6)                # RXX(ip_type * tld)

    # -----------------------
    # Group C semantic entanglement
    qc.append(RZZGate(x[8] * x[9]), [8,9])
    qc.ryy(x[8] * x[9], 8, 9)

    # -----------------------
    # Cross-group entanglement (original)
    qc.append(RZZGate(x[3] * x[6]), [3,6])  # RZZ(Q3 * Q6)
    qc.rxx(x[1] * x[8], 1, 8)                # RXX(Q1 * Q8)

    # -----------------------
    # Optional light global ring to mix everything (keeps some connectivity)
    for i in range(num_qubits - 1):
        qc.cx(i, i+1)

    return qc

# Build the feature_map circuit (parametrized)
feature_map = build_phish_feature_map(params)
feature_map.decompose().draw(output='text')  # optional textual circuit printout


global phase: -π/2*x[1] - π/2*x[5]
     ┌───────────────┐                                            »
q_0: ┤ R(π*x[0],π/2) ├───────────────────■──────────────────■─────»
     └─┬────────────┬┘   ┌───┐         ┌─┴─┐      ┌───┐   ┌─┴─┐   »
q_1: ──┤ U1(π*x[1]) ├────┤ H ├─────────┤ X ├──────┤ H ├───┤ X ├───»
      ┌┴────────────┤    └───┘         └───┘      └───┘   └───┘   »
q_2: ─┤ R(π*x[2],0) ├─────────────────────────────────────────────»
     ┌┴─────────────┴┐┌─────────┐                                 »
q_3: ┤ R(π*x[3],π/2) ├┤ Rx(π/2) ├─────────────────────────────────»
     ├───────────────┤└─────────┘                                 »
q_4: ┤ R(π*x[4],π/2) ├───────────────────■────────────────────────»
     └─┬────────────┬┘   ┌───┐         ┌─┴─┐      ┌───┐           »
q_5: ──┤ U1(π*x[5]) ├────┤ H ├─────────┤ X ├──────┤ H ├─────■─────»
      ┌┴────────────┤    ├───┤         └───┘      └───┘   ┌─┴─┐   »
q_6: ─┤ R(π*x[6],0) ├────┤ H ├────────────────────────────┤ X ├───»
     ┌┴─────────────┴┐   ├───┤                            └───┘   »
q_7: ┤ R(π*x[7],π/2) ├───┤ H ├────────────────────────────────────»
     ├───────────────┤   └───┘                         ┌─────────┐»
q_8: ┤ R(π*x[8],π/2) ├─────■────────────────────────■──┤ Rx(π/2) ├»
     └┬─────────────┬┘   ┌─┴─┐   ┌───────────────┐┌─┴─┐├─────────┤»
q_9: ─┤ R(π*x[9],0) ├────┤ X ├───┤ Rz(x[8]*x[9]) ├┤ X ├┤ Rx(π/2) ├»
      └─────────────┘    └───┘   └───────────────┘└───┘└─────────┘»
«                                                                  »
«q_0: ─────────────────────────■───────────────────────────────────»
«     ┌───────────────┐      ┌─┴─┐                 ┌───┐           »
«q_1: ┤ Rz(x[0]*x[1]) ├──────┤ X ├────────■────────┤ H ├───────────»
«     └───────────────┘      └───┘      ┌─┴─┐   ┌──┴───┴──┐        »
«q_2: ──────────────────────────────────┤ X ├───┤ Rx(π/2) ├─────■──»
«                                       └───┘   └─────────┘   ┌─┴─┐»
«q_3: ────────────────────────────────────────────────────────┤ X ├»
«                                                             └───┘»
«q_4: ────────■───────────────────────────■────────────────────────»
«           ┌─┴─┐      ┌───────────────┐┌─┴─┐                      »
«q_5: ──────┤ X ├──────┤ Rz(x[4]*x[5]) ├┤ X ├──────────────────────»
«           ├───┤      └─────┬───┬─────┘├───┤┌───────────────┐┌───┐»
«q_6: ──────┤ H ├────────────┤ H ├──────┤ X ├┤ Rz(x[6]*x[7]) ├┤ X ├»
«           └───┘            └───┘      └─┬─┘└───────────────┘└─┬─┘»
«q_7: ────────────────────────────────────■─────────────────────■──»
«                                               ┌──────────┐  ┌───┐»
«q_8: ────────■───────────────────────────■─────┤ Rx(-π/2) ├──┤ H ├»
«           ┌─┴─┐      ┌───────────────┐┌─┴─┐   ├──────────┤  └───┘»
«q_9: ──────┤ X ├──────┤ Rz(x[8]*x[9]) ├┤ X ├───┤ Rx(-π/2) ├───────»
«           └───┘      └───────────────┘└───┘   └──────────┘       »
«                                                                  »
«q_0: ─────────────────────────────────────────────────────────────»
«                                                             ┌───┐»
«q_1: ───────────────────■─────────────────────────────────■──┤ H ├»
«                        │                   ┌──────────┐  │  └───┘»
«q_2: ───────────────────┼──────────■────────┤ Rx(-π/2) ├──┼───────»
«     ┌───────────────┐  │        ┌─┴─┐      ├──────────┤  │       »
«q_3: ┤ Rz(x[2]*x[3]) ├──┼────────┤ X ├──────┤ Rx(-π/2) ├──┼────■──»
«     └───────────────┘  │        └───┘      └──────────┘  │    │  »
«q_4: ───────────────────┼─────────────────────────────────┼────┼──»
«                        │                                 │    │  »
«q_5: ───────────────────┼─────────────────────────────────┼────┼──»
«           ┌───┐        │                                 │  ┌─┴─┐»
«q_6: ──────┤ H ├────────┼─────────────────────────────────┼──┤ X ├»
«           ├───┤        │                                 │  └───┘»
«q_7: ──────┤ H ├────────┼───────────────────────────

In [12]:
# CELL 9: QUANTUM KERNEL 

print("\n⚛️ Setting up quantum kernel...")
print("=" * 50)

from qiskit_machine_learning.kernels import FidelityQuantumKernel
from qiskit.primitives import Sampler

# Create a sampler 
sampler = Sampler()  # default: statevector simulator

# Create the quantum kernel with custom feature map
quantum_kernel = FidelityQuantumKernel(
    feature_map=feature_map,
    # sampler=sampler   
)

print(f"✅ FidelityQuantumKernel created successfully")




⚛️ Setting up quantum kernel...
✅ FidelityQuantumKernel created successfully


In [13]:
# CELL 10: TRAIN QSVC

qsvc = QSVC(quantum_kernel=quantum_kernel, C=1.0)

print("Computing kernel matrices and training QSVC ...")
qsvc.fit(X_train_10, y_train)
print("QSVC training finished.")


Computing kernel matrices and training QSVC ...
QSVC training finished.


In [14]:
# CELL 11: EVALUATION
y_pred = qsvc.predict(X_test_10)

print("Accuracy:", accuracy_score(y_test, y_pred))
print("\nConfusion Matrix:\n", confusion_matrix(y_test, y_pred))
print("\nClassification Report:\n", classification_report(y_test, y_pred))


Accuracy: 0.8666666666666667

Confusion Matrix:
 [[25  5]
 [ 3 27]]

Classification Report:
               precision    recall  f1-score   support

           0       0.89      0.83      0.86        30
           1       0.84      0.90      0.87        30

    accuracy                           0.87        60
   macro avg       0.87      0.87      0.87        60
weighted avg       0.87      0.87      0.87        60



In [15]:
# CELL 12: CLASSICAL SVM FOR COMPARISON

print("\n🖥️ Running classical SVM for comparison...")
print("="*50)

# Features to use (same as QSVC)
feature_cols = [
    "email_subject_len","email_has_urgent_keyword","email_url_len","email_num_links_in_email",
    "web_url_len","web_js_len","web_js_obf_len","domain_age",
    "email_from_domain_score","web_tld_score","web_ip_score","web_https_bin","content_len",
    "sem_comp_0","sem_comp_1"
]

X_train = train_df[feature_cols].values
X_test  = test_df[feature_cols].values
y_train = train_df["final_label"].values
y_test  = test_df["final_label"].values

# Standardize features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled  = scaler.transform(X_test)

# Create classical SVM (RBF kernel)
clf = SVC(kernel='rbf', C=1.0, gamma='scale', random_state=SEED)

# Train
clf.fit(X_train_scaled, y_train)

# Predict
y_pred = clf.predict(X_test_scaled)

# Evaluate
acc = accuracy_score(y_test, y_pred)
print(f"\n✅ Classical SVM Accuracy: {acc*100:.2f}%\n")

print("📊 Classification Report:\n")
print(classification_report(y_test, y_pred))

# Confusion matrix
cm = confusion_matrix(y_test, y_pred)
print("Confusion Matrix:\n", cm)



🖥️ Running classical SVM for comparison...

✅ Classical SVM Accuracy: 95.00%

📊 Classification Report:

              precision    recall  f1-score   support

           0       0.91      1.00      0.95        30
           1       1.00      0.90      0.95        30

    accuracy                           0.95        60
   macro avg       0.95      0.95      0.95        60
weighted avg       0.95      0.95      0.95        60

Confusion Matrix:
 [[30  0]
 [ 3 27]]
